# 0. what for
1. 把一张表拆成多个csv文件
# 1. Preparations
## 1.1 全局设置

In [58]:
#导入pandas包
import pandas as pd
import numpy as np
from tqdm import tqdm

#data path
data_np_path = r'E:\DATA\large_dta\auctions_np.csv'                 #
settings_np_path = r'E:\DATA\large_dta\settings_np.csv'             #

# data output path
target_output_head= r"E:\DATA\large_dta\targets\target_data_NP_"
target_output_tail= ".csv"

In [59]:
data = pd.read_csv(data_np_path,encoding="utf-8")
settings = pd.read_csv(settings_np_path,encoding="utf-8")

unique_setting_NN = ['item','k','y','retail']
settings.head()

,item,k,y,retail
0,20 BIDS VOUCHER,0.01,0.01,14.0
1,20 BIDS VOUCHER,0.01,0.01,15.0
2,50 BIDS VOUCHER,0.01,0.01,35.0
3,50 FREEBIDS VOUCHER!,0.01,0.01,35.0
4,MAKITA CORDLESS ARTICULATED WRENCH,0.01,0.01,59.0


# 2. output

In [66]:
START = 1196
for i in range(len(settings)):
    data_i_df = data[(data['item']==settings.iloc[i,0]) & (data['k']==settings.iloc[i,1]) & (data['y']==settings.iloc[i,2]) & (data['retail']==settings.iloc[i,3])]

    data_i_df = data_i_df[['N','P']].copy()
    data_i_df.sort_values(by="N",inplace=True,ignore_index=True)
    # Get the output path
    target_output_path = target_output_head+str(START+i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# 3. 整理粒度
1. read in data （with 'cnt'）
2. 粒度=5

In [64]:
target_output_head= r"E:\DATA\large_dta\targets_5\target_data_NP_"
target_output_tail= ".csv"

In [65]:
for i in tqdm(range(0,len(settings))):

    data_i = data[(data['item']==settings.iloc[i,0]) & (data['k']==settings.iloc[i,1]) & (data['y']==settings.iloc[i,2]) & (data['retail']==settings.iloc[i,3])]
    data_i = data_i[['N','P','cnt_N']].copy()
    data_i.sort_values(by="N",inplace=True,ignore_index=True)

    # 无重复
    data_i_df = data_i.drop_duplicates(inplace = False,ignore_index=True)
    # New added: Rearrange the data according to their length
    data_lenth = data_i_df.shape[0]         # Data length
    dele_idx = []                           # Idx to be deleted

    if(data_lenth >= 0):
        j = 0
        while (j < data_lenth):
            # N值-1恰好是5倍
            if((data_i_df.iloc[j,0]-1)%5 == 0):
                k = j+1    # 用k记录位置
                # k不是data最后一个值 and [k]在+5的范围内
                while((k < data_lenth) and (data_i_df.iloc[j,0]+5 > data_i_df.iloc[k,0])):
                    # 更新[j]的prob值和cnt值：叠加
                    data_i_df.iloc[j,1] += data_i_df.iloc[k,1]
                    data_i_df.iloc[j,2] += data_i_df.iloc[k,2]
                    # 删除[k]
                    dele_idx.append(k)
                    k += 1
                # j 从 k（其实是k+1）的位置继续
                j = k
            # N值不是5倍且未被并入任何一个已存在的开头
            else:
                # 原地修改（减小）成一个新的区间的开头,
                while((data_i_df.iloc[j,0]-1) %5 != 0):
                    data_i_df.iloc[j,0] -= 1
                # 注意不需要j+1，下一次循环从当前开始，check后面的n需不需要并进来
                # j += 1

    ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
    # Save i_th training data file
    # save_i_trainfile(i)
    # Dele
    data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

    # Reconstruct and repeat data according to 'cnt_n_2'
    data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt_N'])]
    data_i_df = data_i_df[['N','P']]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")


100%|██████████| 78/78 [00:01<00:00, 72.42it/s]
